In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('aplicaciones_limpias.csv')
data.info()
data = data.rename(columns={'Aplicación': 'Aplicacion'})
data['nivel'] = data['nivel'].map({1: 'Junior', 0: 'No Junior'})

<class 'pandas.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Aplicación     171 non-null    int64
 1   Empresa        171 non-null    str  
 2   Posicion       171 non-null    str  
 3   Fecha          171 non-null    str  
 4   url            154 non-null    str  
 5   Junta ARC      171 non-null    int64
 6   Canal_limpio   171 non-null    str  
 7   tipo_posicion  171 non-null    str  
 8   nivel          171 non-null    int64
dtypes: int64(3), str(6)
memory usage: 12.2 KB


In [3]:
#Busco hacer IDs únicos para cada empresa y cada aplicación. 
dim_canal = data[['Canal_limpio']].drop_duplicates().reset_index(drop=True)
dim_canal['Canal_ID'] = range(1, len(dim_canal) + 1)

dim_fecha = data[['Fecha']].drop_duplicates().reset_index(drop=True)
dim_fecha['Fecha_ID'] = range(1, len(dim_fecha) + 1)

dim_empresa = data[['Empresa']].drop_duplicates().reset_index(drop=True)
dim_empresa['Empresa_ID'] = range(1, len(dim_empresa) + 1)

dim_aplicacion = data[['Aplicacion']].drop_duplicates().reset_index(drop=True)
dim_aplicacion['Aplicacion_ID'] = range(1, len(dim_aplicacion) + 1)

dim_junta = data[['Junta ARC']].drop_duplicates().reset_index(drop=True)
dim_junta['Junta_ID'] = range(1, len(dim_junta) + 1)

#dim posiciones 
dim_posicion = data[['tipo_posicion', 'nivel']].drop_duplicates().reset_index(drop=True)
dim_posicion['Posicion_ID'] = range(1, len(dim_posicion) + 1)
dim_posicion['nivel'] = dim_posicion['nivel'].map({1: 'Junior', 0: 'No Junior'})

#Renombrar columna con tilde 
dim_aplicacion = dim_aplicacion.rename(columns={'Aplicación': 'Aplicacion'})

display(dim_canal.head())
display(dim_fecha.head())
display(dim_empresa.head())
display(dim_aplicacion.head())
display(dim_posicion.head())


,Canal_limpio,Canal_ID
0,LinkedIn,1
1,Otros,2
2,CT,3
3,Glassdoor,4
4,Indeed,5


,Fecha,Fecha_ID
0,2025-12-05,1
1,2025-12-08,2
2,2025-12-09,3
3,2025-12-10,4
4,2025-12-11,5


,Empresa,Empresa_ID
0,Idis,1
1,Amaris,2
2,Smilepill,3
3,Adam Milo,4
4,Incode,5


,Aplicacion,Aplicacion_ID
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


,tipo_posicion,nivel,Posicion_ID
0,Otros,NaN,1
1,Data Scientist,NaN,2
2,Data Scientist,NaN,3
3,Data Analyst,NaN,4
4,Data Analyst,NaN,5


In [4]:
fact_aplicaciones = (
    data
    .merge(dim_canal, how='left', on='Canal_limpio').merge(dim_fecha, how='left', left_on='Fecha', right_on='Fecha') 
    .merge(dim_empresa, how='left', on='Empresa') 
    .merge(dim_aplicacion, how='left', on='Aplicacion')  
    .merge(dim_junta, how='left', on='Junta ARC')
    .merge(dim_posicion, how='left', on=['tipo_posicion', 'nivel']))

#Seleccionar columnas para tabla de hechos 
hechos_aplicaciones = fact_aplicaciones[['Canal_ID', 'Fecha_ID', 'Empresa_ID', 'Aplicacion_ID', 'Junta_ID', 'Posicion_ID']]

In [5]:

#Retirar columnas de dimensiones 
fact_aplicaciones = fact_aplicaciones.drop(columns=['Canal_limpio', 'Empresa', 'Junta ARC', 'tipo_posicion', 'nivel', 'Posicion'])

#Tabla de hechos. ID único para cada conexión. 
display(fact_aplicaciones)

,Aplicacion,Fecha,url,Canal_ID,Fecha_ID,Empresa_ID,Aplicacion_ID,Junta_ID,Posicion_ID
0,1,2025-12-05,NaN,1,1,1,1,1,NaN
1,2,2025-12-05,NaN,1,1,2,2,1,NaN
2,3,2025-12-05,NaN,1,1,3,3,1,NaN
3,4,2025-12-05,NaN,1,1,4,4,1,NaN
4,5,2025-12-05,https://job-boards.greenhouse.io/incode/jobs/5...,2,1,5,5,2,NaN
...,...,...,...,...,...,...,...,...,...
166,174,2026-01-20,https://www.linkedin.com/jobs/view/4312877556/...,1,21,75,167,5,NaN
167,175,2026-01-22,NaN,1,22,145,168,5,NaN
168,176,2026-01-23,https://jobs.thermofisher.com/global/en/job/R-...,2,23,39,169,5,NaN
169,177,2026-01-23,https://mx.indeed.com/rc/clk?jk=290262b9a0fb75...,5,23,126,170,5,NaN


In [6]:
#Pasar a CSVs exportables para power bi 
dim_canal.to_csv('dim_canal.csv', index=False)
dim_junta.to_csv('dim_junta.csv', index=False)
dim_fecha.to_csv('dim_fecha.csv', index=False) 
dim_empresa.to_csv('dim_empresa.csv', index=False)
dim_aplicacion.to_csv('dim_aplicacion.csv', index=False)
dim_posicion.to_csv('dim_posicion.csv', index=False)
fact_aplicaciones.to_csv('fact_aplicaciones.csv', index=False)
